In [2]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import time
from numpy import matmul as MM
from sklearn.metrics import normalized_mutual_info_score as nmi
import pycuda.driver as drv
import time


start = drv.Event()
end = drv.Event()

In [3]:
def load_dataset(name='seeds'):
    if name == 'seeds':
        with open('datasets/seeds_dataset.txt') as ds:
            lines = ds.readlines()
            X = []
            Y = []
            for line in lines:
                x = line.split()
                y = x[-1]
                x = x[:-1]
                X.append(x)
                Y.append(y)
            X = np.array(X, dtype=np.float)
            Y = np.array(Y, dtype=np.uint8)
            return X, Y
    if name == 'wine':
         with open('datasets/wine_dataset.txt') as ds:
            lines = ds.readlines()
            X = []
            Y = []
            for line in lines:
                x = line.strip().split(',')
                y = x[0]
                x = x[1:]
                X.append(x)
                Y.append(y)
            X = np.array(X, dtype=np.float)
            Y = np.array(Y, dtype=np.uint8)
            return X, Y  
        
    if name == 'soy':
        with open('datasets/soybean_dataset.txt') as ds:
            lines = ds.readlines()
            X = []
            Y = []
            for line in lines:
                x = line.strip().split(',')
                y = x[-1]
                x = x[:-1]
                X.append(x)
                Y.append(int(y[-1]) - 1)
            X = np.array(X, dtype=np.float)
            Y = np.array(Y, dtype=np.uint8)
            return X, Y  
    if name == 'hand':
        with open('datasets/handjob_dataset.txt') as ds:
            lines = ds.readlines()
            X = []
            Y = []
            for line in lines:
                x = line.strip().split(',')
                y = x[0]
                x = x[1:]
                X.append(x)
                Y.append(y)
            X = np.array(X, dtype=np.float)
            Y = np.array(Y, dtype=np.uint8)
            Y = Y if np.min(Y) == 0 else Y-1
            return X, Y  
    if name == 'olive':
        with open('datasets/olive_dataset.txt') as ds:
            lines = ds.readlines()
            X = []
            Y = []
            for line in lines:
                x = line.strip().split(',')
                y = x[0]
                x = x[1:]
                X.append(x)
                Y.append(y)
            X = np.array(X, dtype=np.float)
            Y = np.array(Y, dtype=np.uint8)
            Y = Y if np.min(Y) == 0 else Y-1
            return X, Y
    if name == 'symbol':
        with open('datasets/symbol_dataset.txt') as ds:
            lines = ds.readlines()
            X = []
            Y = []
            for line in lines:
                x = line.strip().split(',')
                y = x[0]
                x = x[1:]
                X.append(x)
                Y.append(y)
            X = np.array(X, dtype=np.float)
            Y = np.array(Y, dtype=np.uint8)
            Y = Y if np.min(Y) == 0 else Y-1
            return X, Y  
    if name == 'plane':
        with open('datasets/plane_dataset.txt') as ds:
            lines = ds.readlines()
            X = []
            Y = []
            for line in lines:
                x = line.strip().split(',')
                y = x[0]
                x = x[1:]
                X.append(x)
                Y.append(y)
            X = np.array(X, dtype=np.float)
            Y = np.array(Y, dtype=np.uint8)
            Y = Y if np.min(Y) == 0 else Y-1
            return X, Y  
    if name == 'mnist':
        with open('datasets/mnist_dataset.txt') as ds:
            lines = ds.readlines()
            X = []
            Y = []
            for line in lines:
                x = line.strip().split(',')
                y = x[0]
                x = x[1:]
                X.append(x)
                Y.append(int(float(y)))
            X = np.array(X, dtype=np.float)
            Y = np.array(Y, dtype=np.uint8)
            Y = Y if np.min(Y) == 0 else Y-1
            return X, Y  

In [4]:
def normalize_dataset(X):
    n, d = X.shape
    for i in range(d):
        mu = np.mean(X[:, i])
        std = np.std(X[:, i]) + 1e-10
        X[:, i] = (X[:, i] - mu)/std
    return X

In [5]:
def random_V(d):
    
    
    V = np.random.uniform(0, 1, [d, d])
    
    
    start.record()
    start.synchronize()
    
    V = LA.qr(V, mode='complete')[0]
    
    end.record()
    end.synchronize()
    print("Orthonormal time cpu: ",start.time_till(end)*1e-3)
    
    return V

In [6]:
def projection_matrices(d, m):
    Pc = np.zeros([d, m], dtype=np.uint8)
    Pc[:m, :m] = np.eye(m, dtype=np.uint8)
    # Pn = Pc ^ np.eye(d, dtype=np.uint8)
    return Pc

In [7]:
def sub_kmeans(X, k):
    
    n, d = X.shape
    V = random_V(d)
    m = d/2
    
    # Algorithm line 5
    #################################################################################
    start = drv.Event()
    end = drv.Event()
    start.record()
    start.synchronize()
    
    
    mu_D = np.mean(X, axis=0, keepdims=True)
    
    end.record()
    end.synchronize()
    print("Mu_d cpu: ",start.time_till(end)*1e-3)
    #################################################################################
    
    # Algorithm line 6
    #################################################################################
    start.record()
    start.synchronize()
    
    S_D = MM((X - mu_D).T, (X - mu_D)) # Check this using loop
    
    end.record()
    end.synchronize()
    print("S_d cpu: ",start.time_till(end)*1e-3)
    #################################################################################
    
    mu_is = X[np.random.choice(n, k)]
    itr = 1
    assignment_unchanged = 0
    while True:
        
        Pc = projection_matrices(d, m)
        
        # Algorithm line 11
        #################################################################################
        start.record()
        start.synchronize()
        
        PcV = MM(Pc.T, V.T)[None, :, :] # 1,m,d
        PcVmu_is = MM(PcV, mu_is[:, :, None]) # k,m,1
 
        X_trans = MM(PcV.squeeze(0), X.T).T # n, m
        Mus_trans = PcVmu_is.squeeze(-1) # k, m
        sq_diff = np.square(X_trans[:, None, :] - Mus_trans[None, :, :]) # n, k, m
        sq_diff = np.sum(sq_diff, axis=-1)

        
        if itr % 2 == 0:
            Cnew = np.argmin(sq_diff, axis=1)
            points_changed = np.sum(1 - np.equal(C, Cnew).astype(np.uint8))
            if points_changed == 0:
                assignment_unchanged += 1
            if assignment_unchanged >= 2:
                break
            print('\nIter %d: %d points changed' % (itr, points_changed))
            C = Cnew
            # print(set(C))
        else:
            print("\n")
            C = np.argmin(sq_diff, axis=1)
            
        end.record()
        end.synchronize()
        print("Argmin cpu: ",start.time_till(end)*1e-3)
        #################################################################################    
            
        
        # Algorithm line 14
        #################################################################################
        start.record()
        start.synchronize()
        
        counts = {i:0 for i in range(k)}
        mu_is = np.zeros([k, d])
        S_is = np.zeros([k, d, d])
        for i, x in enumerate(X):
            c_id = C[i]
            mu_is[c_id] += x
            counts[c_id] += 1

        mu_is = np.array([mu_is[i]/counts[i] for i in range(k)])
        
        end.record()
        end.synchronize()
        print("New_mu cpu: ",start.time_till(end)*1e-3)
        #################################################################################    
            
        
        
        # Algorithm line 15
        #################################################################################
        start.record()
        start.synchronize()
                
        C_matrics = {i:[] for i in range(k)}

        for i, x in enumerate(X):
            c_id = C[i]
            x_minus_mu_isi = (x - mu_is[c_id]).T[:, None]
            C_matrics[c_id].append(x_minus_mu_isi)
        for ki in C_matrics:
            CX_m = np.array(C_matrics[ki]).squeeze()
            S_is[ki] = MM(CX_m.T, CX_m)
        
        end.record()
        end.synchronize()
        print("New_s_i cpu: ",start.time_till(end)*1e-3)
        #################################################################################    
            
        
        # Algorithm line 16
        #################################################################################
        start.record()
        start.synchronize()
        
        Evals, Evecs = LA.eig(np.sum(S_is, axis=0) - S_D)
        
        end.record()
        end.synchronize()
        print("Eigen_decomp cpu: ",start.time_till(end)*1e-3)
        #################################################################################    
          
        idx = np.argsort(Evals)
        V = Evecs[:, idx]
        maxVal = min(Evals)
        if itr > 2:
            m = np.sum([1 for i in Evals if i/maxVal > 1e-8])
            m = max(1, m)
        
        itr += 1
        
    return C, V, m

In [9]:
X, Y = load_dataset('mnist')
X = normalize_dataset(X)
for i in range(1):
    C, V, m = sub_kmeans(X, 10)
    Pc = projection_matrices(X.shape[1], m)
    trans = V.T
    X_rotated = MM(trans[None, :, :], np.transpose(X[:, None, :], [0, 2, 1]))
    X_rotated = X_rotated.squeeze(-1).T
    acc = nmi(Y, C)
    # print(acc)
    print(m, acc)
    plt.scatter(X_rotated[0], X_rotated[1], c=C)
    plt.show()

Orthonormal time cpu:  0.125591842651
Mu_d cpu:  0.00740412807465
S_d cpu:  0.374516265869


Argmin cpu:  0.518779541016
New_mu cpu:  0.0353199691772
New_s_i cpu:  0.309780853271
Eigen_decomp cpu:  2.65453540039

Iter 2: 2723 points changed
Argmin cpu:  1.32084692383
New_mu cpu:  0.0307662086487
New_s_i cpu:  0.319939025879


KeyboardInterrupt: 

In [ ]:
for i in range(9):
    for j in range(i+1, 9):
        plt.scatter(X_rotated[i], X_rotated[j], c=C)
        plt.show()

In [ ]:
X_rotated.shape

In [ ]:
X_scala = np.loadtxt('/home/abdul/Documents/adm-project/SubKmeans/result/sample_result.dat', delimiter=';')

In [ ]:
X_scala.shape

In [ ]:
C = X_scala[:, 0]
X_scala = X_scala[:, 1:]

In [ ]:
nmi(Y, C)

In [ ]:
plt.scatter(X_scala[:, 0], X_scala[:, 1], c=C)
plt.show()

In [ ]:
C

In [ ]:
X, Y = load_dataset('wine')
X = normalize_dataset(X)


In [ ]:
with open('/home/abdul/Documents/adm-project/SubKmeans/datasets/sample.dat', 'w') as f:
    for i, x in enumerate(X):
        f.write(str(Y[i]))
        f.write(';')
        f.write(';'.join(['%.15f'%j for j in x]))
        f.write('\n')

In [ ]:
def sub_kmeans_loopy(X, k):
    n, d = X.shape
    V = random_V(d)
    m = d/2
    mu_D = np.mean(X, axis=0, keepdims=True)
    S_D = np.zeros([d, d])
    
    Pc = projection_matrices(d, m)
    for e, x in enumerate(X):
        x_minus_muD = (x - mu_D).T
        S_D += MM(x_minus_muD, x_minus_muD.T)
    mu_is = X[np.random.choice(n, k)]
    itr = 0
    while itr < 200:
        C = {i:[] for i in range(k)}
        ass = np.zeros(n, dtype=np.uint8) - 1
        Pc = projection_matrices(d, m)
        trans = MM(Pc.T, V.T)
        for k_num, x in enumerate(X):
            j = -1
            min_dist = 1e10
            for i, mu_i in enumerate(mu_is):
                vec_dist = MM(trans, x[:, None]) -  MM(trans, mu_i[:, None])
                dist = MM(vec_dist.T, vec_dist)[0, 0]
                if dist < min_dist:
                    min_dist = dist
                    j = i
            C[j].append(x)
            ass[k_num] = j
        S = np.zeros([k, d, d])
        for i in range(k):
            X_in_Ci = np.array(C[i])
            mu_is[i] = np.mean(X_in_Ci, axis=0)
            for x in X_in_Ci:
                x_minus_mu_isi = (x - mu_is[i]).T[:, None]
                S[i] += MM(x_minus_mu_isi, x_minus_mu_isi.T)
        sum_S = np.sum(S, axis=0)
        Evals, Evecs = LA.eig((sum_S - S_D))
#         print(Evals.shape)
        idx = np.argsort(Evals)
#         print(Evals.shape)
        V = Evecs[:, idx]
        maxVal = min(Evals)
        if itr > 1:
            m = np.sum([1 for i in Evals if i/maxVal > 1e-8])
            m = max(1, m)
        itr += 1
        ccc = nmi(Y, ass)
    return ass, V, m
        
            

In [ ]:
import tensorflow as tf
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

In [ ]:
data = np.concatenate([mnist.test.labels[:, None], mnist.test.images], axis=1)
np.savetxt('datasets/mnist_dataset.txt', data, delimiter=',', fmt='%.8f')